In [1]:
%pylab
from numba import jit, njit

Using matplotlib backend: TkAgg
Populating the interactive namespace from numpy and matplotlib


In [2]:
softening = 0.

@njit(fastmath=True)
def accel(dx):
    r = 0
    for k in range(3):
        r += dx[k]*dx[k]
    r = np.sqrt(r)

    fac = gravfac(r)#1/(r*r*r)
    #else: fac = 1/(r*r*r)

    return -dx * fac

@njit(fastmath=True)
def gravfac(r):
    if r < softening:
        u = r/softening
        if u<0.5:
            fac = (10.666666666667 + u * u * (32.0 * u - 38.4)) / softening**3
        else:
            fac = (21.333333333333 - 48.0 * u + 38.4 * u * u - 10.666666666667 * u * u * u - 0.066666666667 / (u * u * u))/softening**3
    else:
        fac = 1/(r*r*r)
    return fac

@njit(fastmath=True)
def gravfac2(r):
    if r < softening:
        u = r/softening
        if u<0.5:
            fac = (76.8 - 96.0 * u) / softening**5
        else:
            fac = (-0.2 / (u * u * u * u * u) + 48.0 / u - 76.8 + 32.0 * u) / softening**5
    else:
        fac = 3/r**5
    return fac

@vectorize
def potential(r):
    if r < softening:
        u = r / softening
        if(u < 0.5):
            wk = -2.8 + u * u * (5.333333333333 + u * u * (6.4 * u - 9.6));
        else:
            wk = -3.2 + 0.066666666667 / u + u * u * (10.666666666667 +
                                                      u * (-16.0 + u * (9.6 - 2.133333333333 * u)));
        return wk/softening
    else:
        return -1/r



@njit(fastmath=True)
def jerk(dx, dv):
    r = 0
    for k in range(3):
        r += dx[k]*dx[k]
    r = np.sqrt(r)
    
    if softening > 0:
        fac = gravfac(r)
        fac2 = gravfac2(r) #fac / (r*r)
    else:
        fac = 1/(r*r*r)
        fac2 = 3*fac / (r*r)
    
    dv_dot_dx = dx[0]*dv[0] + dx[1]*dv[1] + dx[2]*dv[2]
    return -dv * fac + dv_dot_dx * dx * fac2
    


    
def Energy(pos, vel):
    return 0.5*np.sum(vel**2,axis=1) + potential(np.sqrt(np.sum(pos**2,axis=1)))
    #return acc

In [391]:
plt.plot(np.linspace(0,1,1000), [gravfac(i) for i in np.linspace(0,1,1000)])

In [73]:
#Leapfrog scheme

def LeapfrogStep(pos, vel, dt):
    vel[:] += accel(pos) * dt/2
    pos[:] += vel * dt
    vel[:] += accel(pos) * dt/2
    return

def LeapfrogJerkStep(pos, vel, acc, dt, coeffs):
    j0 = jerk(pos, vel-accel(pos)*dt/2)
    acc[:] += j0 * dt/2
    vel[:] += acc * dt/2 #+ j0 * dt*dt/8
    #j0 = jerk(pos, vel)
    #j0 = jerk(pos, vel)
    #vel[:] += accel(pos) * dt/2# + j0 * dt*dt/24
    #j0 = jerk(pos, vel)
    #vel[:] += j0*dt*dt/24
    pos[:] += vel * dt #+ jerk(pos,vel) * dt**3 / 6
    j0 = jerk(pos, vel)
    #j1 = jerk(pos, vel)
    #vel[:] += j1*dt*dt/24
    #j0 = jerk(pos, vel)
    #a = accel(pos) + j0 * dt/2
    #j0 = jerk(pos, vel)
    #a = accel(pos) + j0 * dt/2
    vel[:] += accel(pos) * dt/2# + j0 * dt*dt/8
    acc[:] += j0 * dt/2
    return


@jit
def RK4(pos, vel, dt):
    v0 = np.copy(vel)
    x0 = np.copy(pos)
    
    dx1 = dt * v0
    dv1 = dt * accel(x0)
    dx2 = dt * (v0 + dv1/2)
    dv2 = dt * accel(x0 + dx1/2)
    dx3 = dt * (v0 + dv2/2)
    dv3 = dt * accel(x0 + dx2/2)
    dx4 = dt * (v0 + dv3)
    dv4 = dt * accel(x0 + dx3)
    
    pos[:] = x0 + dx1/6 + dx2/3 + dx3/3 + dx4/6
    vel[:] = v0 + dv1/6 + dv2/3 + dv3/3 + dv4/6
    
@jit 
def Hermite(pos, vel, dt):
    v0 = np.copy(vel)
    x0 = np.copy(pos)
    a0 = accel(x0)
    j0 = jerk(x0, v0)
    
    dt2 = dt*dt
    dt3 = dt*dt2
    dt4 = dt*dt3
    # prediction - do this in first half-step kick in GIZMO
    dx1 = v0 * dt + 0.5 * a0 * dt2 + j0 * dt3/6
    dv1 = a0 * dt + 0.5 * j0 * dt2
    
    pos[:] += dx1
    vel[:] += dv1
    
    a1 = accel(pos)
    j1 = jerk(pos, vel)
    
    # correction - do this in second half-step kick
    vel[:] = v0 + (a0+a1)*dt/2 + (j0-j1)*dt2/12 #dx1 + dt4/24 * s0 + (dt*dt4)/120 * c0
    pos[:] = x0 + (vel + v0)*dt/2 + (a0 - a1)*dt2/12
    
    return 

    
@jit 
def PoorMansHermite(pos, vel, dt):
    v0 = np.copy(vel)
    x0 = np.copy(pos)
    a0 = accel(x0)
    #j0 = jerk(x0, v0)
    
    dt2 = dt*dt
    dt3 = dt*dt2
    dt4 = dt*dt3
    # prediction - do this in first half-step kick in GIZMO
    dx1 = v0 * dt + 0.5 * a0 * dt2 #+ j0 * dt3/6
    dv1 = a0 * dt #+ 0.5 * j0 * dt2
    
    pos[:] += dx1
    vel[:] += dv1
    
    a1 = accel(pos)
    #j1 = jerk(pos, vel)
    
    # correction - do this in second half-step kick
    vel[:] = v0 + (a0+a1)*dt/2 #+ (j0-j1)*dt2/12 #dx1 + dt4/24 * s0 + (dt*dt4)/120 * c0
    pos[:] = x0 + (vel + v0)*dt/2 #+ (a0 - a1)*dt2/12
    
    return 

In [387]:
Hermite(np.array([1.,0.,0.]), np.array([0.,1.,0.]),1e-4)

In [48]:
softening = 0.
e = 0.9
vel = (1-e)**0.5 * np.array([0,1.,0])
pos = np.array([1.,0.,0.])
period = 2*np.pi * (1-e/2)**1.5
x = [np.copy(pos),]
v = [np.copy(vel),]
a = []
t = 0.
times = [t,]
#dt = 0.00125/2
N_orbits = 1000
tmax = N_orbits*2.4
while t < tmax:
    #omega = np.sqrt(np.sum(np.cross(vel, pos)**2))/np.sum(pos**2)
    #omega = np.sum(vel**2)**0.5 /  np.sum(pos**2)**0.5
    omega = np.sum(vel**2)**0.5 /  np.sum(pos**2)**0.5 + np.sqrt(softening + np.sum(pos**2))**-1.5
    #omega = np.sqrt(softening + np.sum(pos**2))**-1.5
    #omega = np.sqrt(np.sum(vel**2) /  np.sum(pos**2) + np.sqrt(np.sum(pos**2))**-3)
    #print(omega)
    dt = min(0.1/ omega, tmax-t)
    #dt = (0.1 * accel(pos)*)
    #print(pos,vel)
    #LeapfrogStep(pos, vel, dt)
    #RK4(pos, vel, dt)
    LeapfrogJerkStep(pos, vel, dt)
    x.append(np.copy(pos))
    v.append(np.copy(vel))
    t += dt
    times.append(t)
print(tmax)
x = np.array(x)
v = np.array(v)
#print(x[0],v[0])
E = Energy(x,v)
relative_error = np.abs(E-E[0])/np.abs(E[0])
print(len(E)/N_orbits, relative_error.max()/N_orbits * 10**5, relative_error.max()*(len(E)/N_orbits/100)**5)#*(len(E)/1e5)**5, relative_error.max()*1000)
#print(relative_error[-1])
plt.plot(times, relative_error)
#plt.yscale('log')
#plt.xscale('log')
plt.show()
#print(relative_error.max())
#plt.plot(x[:,0], x[:,1])
#plt.axes().set_aspect('equal')
#plt.show()

2400.0
12.703 18137656.911776725 5.99946705820399


In [74]:
def EnergyError(fac):
    print(fac)
    softening = 0.
    e = 0.9
    vel = (1-e)**0.5 * np.array([0,1.,0])
    pos = np.array([1.,0.,0.])
    acc = accel(pos)
    j = jerk(pos, vel)
    period = 2*np.pi * (1-e/2)**1.5
    x = [np.copy(pos),]
    v = [np.copy(vel),]
    t = 0.
    times = [t,]
    #dt = 0.00125/2
    N_orbits = 10
    tmax = N_orbits*2.4
    while t < tmax:
        #omega = np.sqrt(np.sum(np.cross(vel, pos)**2))/np.sum(pos**2)
        #omega = np.sum(vel**2)**0.5 /  np.sum(pos**2)**0.5
        omega = np.sum(vel**2)**0.5 /  np.sum(pos**2)**0.5 + np.sqrt(softening + np.sum(pos**2))**-1.5
        #omega = np.sqrt(softening + np.sum(pos**2))**-1.5
        #omega = np.sqrt(np.sum(vel**2) /  np.sum(pos**2) + np.sqrt(np.sum(pos**2))**-3)
        #print(omega)
        dt = min(fac/ omega, tmax-t)
        #dt = (0.1 * accel(pos)*)
        #print(pos,vel)
        #LeapfrogStep(pos, vel, dt)
        #RK4(pos, vel, dt)
        PoorMansHermite(pos, vel, dt)

        x.append(np.copy(pos))
        v.append(np.copy(vel))
        t += dt
        times.append(t)
    #print(tmax)
    x = np.array(x)
    v = np.array(v)
    #print(x[0],v[0])
    E = Energy(x,v)
    relative_error = np.abs(E-E[0])/np.abs(E[0])
#    print(len(E)/N_orbits, relative_error.max()/N_orbits * 10**5, relative_error.max()*(len(E)/N_orbits/100)**5)#*(len(E)/1e5)**5, relative_error.max()*1000)
    return times, E, relative_error
#print(relative_error[-1])
facs = 0.01*2.**(-np.arange(0,5))
errors = [(EnergyError(f)[2]).max() for f in facs]
plt.plot(facs, errors)
#plt.plot(times, relative_error)
plt.yscale('log')
plt.xscale('log')
plt.show()
print("Method order: ", -np.log10(errors[-1]/errors[0])/np.log10(facs[-1]/facs[0]))

0.01
0.005
0.0025
0.00125
0.000625
Method order:  -2.0610523489382833


In [116]:
print(np.sum(x**2,axis=1).min()**0.5)

0.05263157761208886


In [17]:
np.log10(errors[-1]/errors[0])/np.log10(facs[-1]/facs[0])

2.034717068616203